## History Quiz

Use of LangChain to create a History Quiz Bot

* Writing a historical question that has a date as the correct answer
* Getting the correct answer from LLM
* Getting a Human user's best guess at at correct answer
* Checking/reporting the difference between the correct answer and the user answer

In [27]:
import os 
from datetime import datetime

from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from datetime import datetime
from langchain.llms import OpenAI
from langchain.output_parsers import DatetimeOutputParser
from langchain.chat_models import ChatOpenAI

In [28]:
# DELETE THIS CELL AFTER
os.environ['OPENAI_KEY'] = 'openai_key'

In [83]:
class HistoryQuiz():
    

    def __init__(self) -> None:
        self.api_key = os.getenv('OPENAI_KEY')
        self.model = ChatOpenAI(openai_api_key=api_key)


    def create_history_question(self,topic):
        '''
        This method should output a historical question about the topic that 
        has a date as the correct answer.
        For example:
        
            "On what date did World War 2 end?"
            
        '''
        # PART ONE: SYSTEM
        system_template = """ You are a history expert in {topic}. You are 
                          familiar with all persons and events of {topic}
                          and you know all the dates of the important events.

                          Follow the instructions:
                          - The response should include only the question
                          - The correct answer should be a date
                          """ 
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

        # PART TWO: HUMAN REQUEST
        human_template="""Give me a quiz question about {topic}"""

        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
       
         # PART THREE: COMPILE TO CHAT
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, 
                                                        human_message_prompt])
        
        # PART FOUR: INSERT VARIABLES
        request = chat_prompt.format_prompt(topic=topic).to_messages()
        # PART FIVE: CHAT REQUEST
        
        self.question = self.model(request, temperature=0.7)
        
        return self.question.content
    
    def get_AI_answer(self):
        '''
        This method should get the answer to the historical question from the method above.
        Note: This answer must be in datetime format! Use DateTimeOutputParser to confirm!
        
        September 2, 1945 --> datetime.datetime(1945, 9, 2, 0, 0)
        '''
        output_parser = DatetimeOutputParser()

        system_prompt = SystemMessagePromptTemplate.from_template("You always reply to questions only in datetime patterns.")

        template_text = "{request}\n{format_instructions}"
        human_prompt=HumanMessagePromptTemplate.from_template(template_text)
        
        chat_prompt = ChatPromptTemplate.from_messages([system_prompt,human_prompt])

        request = chat_prompt.format_prompt(request=self.question,
                   format_instructions=output_parser.get_format_instructions()
                   ).to_messages()
        
        result = self.model(request,temperature=0)

        try:
            self.correct_datetime = output_parser.parse(result.content)
        except: 
            return "The result was not in a datetime format:" + self.correct_datetime 
        
        return self.correct_datetime


    def get_user_answer(self):
        '''
        This method should grab a user answer and convert it to datetime. 
        It should collect a Year, Month, and Day.
        You can just use input() for this.
        '''
        
        year = int(input("Enter the year: "))
        while year < 0 and year == '': 
            year = int(input("Enter the year: "))

        month = int(input("Enter the month: "))
        while month > 12 or month < 1 or month == '': 
            month = int(input("Enter the month: "))
            
        day = int(input("Enter the day: "))
        while day > 31 or day < 1 or day == '': 
            day = int(input("Enter the day: "))
        
        self.user_datetime = datetime(year, month, day)
        return self.user_datetime

        
    def check_user_answer(self,user_answer, ai_answer):
        '''
        Should check the user answer against the AI answer and return the 
        difference between them
        '''
        # print or return the difference between the answers here!
        difference = user_answer - ai_answer
        self.reply = f"The difference is {difference.days} dates"
        return self.reply

        
        

In [84]:
quiz_bot = HistoryQuiz()
question = quiz_bot.create_history_question(topic='World War 2')
print(question)

When did the United States enter World War II?


In [86]:
ai_answer = quiz_bot.get_AI_answer()
print(ai_answer)

1941-12-07 00:00:00


In [71]:
user_answer = quiz_bot.get_user_answer()
user_answer

datetime.datetime(1932, 12, 2, 0, 0)

In [72]:
result = quiz_bot.check_user_answer(user_answer,ai_answer)
print(result)

The difference is -2464 dates
